# Dependencies

## Imports

In [2]:
# imports 
# base python
import os
import copy
from sys import getsizeof

# scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d, interp2d
from sklearn import preprocessing
import matplotlib.tri as tri

plt.rcParams["figure.figsize"] = (20,7)

#ase
from ase.io import gen, vasp, xyz, extxyz, dftb
from ase.io.dftb import read_dftb_velocities, write_dftb_velocities
from ase.calculators.dftb import Dftb
from ase import Atoms, Atom
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.build import make_supercell
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
import nglview
from ase.geometry.analysis import Analysis

#dscribe
from dscribe.descriptors import SOAP
from dscribe.descriptors import MBTR
from dscribe.kernels import REMatchKernel
from dscribe.kernels import AverageKernel

from sklearn import preprocessing


#quippy 
from ase.build import bulk
from ase.optimize import LBFGS
from ase.visualize import view
from quippy.potential import Potential


#misc
import similaritymeasures


_ColormakerRegistry()

## Functions

In [3]:
def show_atoms_grid(data, rotation = '-0x,0y,0z', save= False, filename = 'grid_configs'):
    '''
    Where data is list of Atoms objects
    '''
    dim = int(np.ceil(np.sqrt(len(data))))
    fig, axarr = plt.subplots(dim, dim, figsize=(25, 25))
    for i, config in enumerate(data):
        plot_atoms(config, axarr[i%dim,i//dim], rotation = rotation)
    if save:
        fig.savefig(filename + ".png")
        
def normalize(y,x):
    """
    Takes y, x of data and returns normalized y
    """
    return y/np.trapz(y,x)

def KE(v_tot):
    "Returns KE of Ar+ in eV given total velocity"
    return 6.24E18 * 0.5 * 1.66E-27*39.95*(v_tot*1E5)**2

def v_from_KE(E):
    "Returns v(z) of Ar+ in eV given KE"
    return np.sqrt(E/(6.24E18 * 0.5 * 1.66E-27*39.95))/1E5

## Structures

In [4]:
mef = vasp.read_vasp("reference_files/CONTCAR_mef")
cf4 = vasp.read_vasp("reference_files/CONTCAR_cf4")
amorphous = vasp.read_vasp("reference_files/CONTCAR_amorphous_cubic")
xtl_n = vasp.read_vasp("reference_files/CONTCAR_nrich")
xtl_si = vasp.read_vasp("reference_files/CONTCAR_sirich")
xtl_si_fterm = vasp.read_vasp("reference_files/CONTCAR_sirich_fterm")
xtl2x2 = gen.read_gen("reference_files/2x2xtl.gen")
heavy_bomb = vasp.read_vasp("reference_files/CONTCAR_heavy_bombard")

# Fitting 

Tutorial from here:  https://libatoms.github.io/GAP/gap_si_surface.html

## Generate data

Build silicon structure:

In [5]:
from ase import Atoms, units
from ase.lattice.cubic import Diamond
from ase.build import add_vacuum
def build_slab(size=(1,2,2), vacuum=10.):
    # Build Si lattice.
    # lattice = Diamond('Si', directions=([1, 0, 0], [0, 1, 0], [0, 0, 1]), latticeconstant=5.44, size=size)
    lattice = Diamond('Si', latticeconstant=5.44, size=size)
    atoms = Atoms(lattice)


    # Fixing the bottom layer
    bottom = atoms.positions[:,2].min()
    fixed_mask = (abs(atoms.positions[:,2] - bottom) < 2.0)
    atoms.set_constraint(FixAtoms(mask=fixed_mask))

    # build surface by adding space to z direction
    add_vacuum(atoms, vacuum)
    # atoms.center(vacuum=10.0, axis=2)

    return atoms


atoms = build_slab()
print('Number of atoms:', len(atoms))

Number of atoms: 32


Run dynamics, verlet

In [6]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

atoms = build_slab()

# view(atoms)

T = 1000.0 # Temperature [Kelvin]
timestep = 1.0 * units.fs

# attach tight binding calculator

MaxwellBoltzmannDistribution(atoms, 2.0 * T * units.kB)

write_dftb_velocities(atoms, 'velocities.txt')

calculator_quip = Dftb(label='mef',
                      atoms=atoms,
                      run_manyDftb_steps=True,
                      Hamiltonian_MaxAngularMomentum_='',
                      Hamiltonian_MaxAngularMomentum_Si='"p"',
                      kpts = (1, 1, 1),
                      Driver_='VelocityVerlet',
                      Driver_MDRestartFrequency=1,
                      Driver_Velocities_='',
                      Driver_Velocities_empty='<<+ "velocities.txt"',
                      Driver_Steps=1,
                      Driver_KeepStationary='Yes',
                      Driver_TimeStep=0.413413733365614E+02,
                        Driver_Thermostat_='Berendsen',
                        Driver_Thermostat_Temperature=1000* 0.316681534524639E-05,  # 800*1.5 deg Celcius
                        Driver_Thermostat_CouplingStrength=0.001,
                      )


def print_status():
    print('Step = {}, time = {} [fs], T = {} [K]'.format(
        dynamics.nsteps,
        dynamics.nsteps * dynamics.dt / units.fs,
        atoms.get_kinetic_energy() / (1.5 * units.kB * len(atoms))
    ))

def print_energy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    

In [7]:
write_dftb_velocities(atoms, 'velocities.txt')
os.system('rm md.log.* md.out* geo_end*xyz')
atoms.set_calculator(calculator_quip)
dynamics = VelocityVerlet(atoms, timestep)
# dynamics.attach(print_status, interval=1)
# dynamics.attach(print_energy, interval=20)
# dynamics.run(steps = 1)  # run NVE ensemble using DFTB's own driver

In [8]:
db = []
def collect_data():
    db.append(atoms.copy())
    db[-1].info = {'energy':atoms.get_potential_energy()}

dynamics.attach(collect_data, interval=1)
dynamics.run(1000)

from ase.io import write
write('/tmp/atoms_db1.xyz', db)

Run dynamics, DFTB native

In [9]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

atoms = build_slab()

# view(atoms)

T = 1000.0 # Temperature [Kelvin]
timestep = 1.0 * units.fs

# attach tight binding calculator

MaxwellBoltzmannDistribution(atoms, 2.0 * T * units.kB)

write_dftb_velocities(atoms, 'velocities.txt')

calculator_quip = Dftb(label='mef',
                      atoms=atoms,
                      run_manyDftb_steps=True,
                      Hamiltonian_MaxAngularMomentum_='',
                      Hamiltonian_MaxAngularMomentum_Si='"p"',
                      kpts = (1, 1, 1),
                      Driver_='VelocityVerlet',
                      Driver_MDRestartFrequency=1,
                      Driver_Velocities_='',
                      Driver_Velocities_empty='<<+ "velocities.txt"',
                      Driver_Steps=1000,
                      Driver_KeepStationary='Yes',
                      Driver_TimeStep=0.413413733365614E+02,
                        Driver_Thermostat_='Berendsen',
                        Driver_Thermostat_Temperature=1000* 0.316681534524639E-05,  # 800*1.5 deg Celcius
                        Driver_Thermostat_CouplingStrength=0.001,
                      )
    

write_dftb_velocities(atoms, 'velocities.txt')
os.system('rm md.log.* md.out* geo_end*xyz')
atoms.set_calculator(calculator_quip)
dynamics = VelocityVerlet(atoms, timestep)

db = []
def collect_data():
    db.append(atoms.copy())
    db[-1].info = {'energy':atoms.get_potential_energy()}

dynamics.attach(collect_data, interval=1)
dynamics.run(1)




True

In [ ]:
db2 = [i for i in xyz.read_xyz("geo_end.xyz",index = slice(0, -1))]

## Fit and use GAP

In [ ]:
from quippy.descriptors import Descriptor

desc = Descriptor("soap cutoff=4 l_max=3 n_max=4 normalize=T atom_sigma=0.5 n_Z=1 Z={14} ")


In [ ]:
isolated_atom = Atoms("Si", positions=[[0,0,0]])

calc = Dftb(label='Si', atoms=isolated_atom,
            run_manyDftb_steps=True,
            Driver_='ConjugateGradient',
            Driver_MaxForceComponent='1E-4',
            Driver_MaxSteps=1000,
            Hamiltonian_MaxAngularMomentum_='',
            Hamiltonian_MaxAngularMomentum_Si='"p"',)


isolated_atom.set_calculator(calc)
E0 = isolated_atom.get_potential_energy()
E0

In [ ]:
!/home/erik/QUIP/build/linux_x86_64_gfortran/gap_fit --help

In the example below, the .xyz file referenced needs to be in the extended xyz format with an "Energy" field.

In [ ]:
!/home/erik/QUIP/build/linux_x86_64_gfortran/gap_fit at_file=/tmp/atoms_db1.xyz \
gap={distance_Nb order=2 \
                 cutoff=5.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=15 \
                 delta=1.0:\
     distance_Nb order=3 \
                 cutoff=4.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=50 \
                 delta=0.004} \
e0=-29.716948405885105 \
default_sigma={0.005 0.5 0.0 0.0} \
do_copy_at_file=F sparse_separate_file=F \
gp_file=/tmp/gap_3b.xml

In [ ]:
gap = Potential(param_filename='/tmp/gap_3b.xml')

In [ ]:
db = [i for i in xyz.read_xyz("/tmp/atoms_db1.xyz",index = slice(0, -1))]
qm_energies = [at.info['energy'] for at in db]
gap_energies = []
for dba in db:
    a = dba.copy()  
    a.set_calculator(gap)
    gap_energies.append(a.get_potential_energy())

In [ ]:
Natoms = len(db[0])
plt.scatter(np.array(qm_energies)/Natoms, np.array(gap_energies)/Natoms)
plt.show()

In [ ]:
np.sqrt(sum((np.array(qm_energies)/Natoms - np.array(gap_energies)/Natoms)**2)/len(gap_energies))


In [ ]:
dimer_curve = []
for dim in dimers:
    dim.set_calculator(gap)
    dimer_curve.append(dim.get_potential_energy())
plt.plot([dim.positions[1,0] for dim in dimers], np.array(dimer_curve)/2.0)
plt.show()

In [ ]:
gap={distance_Nb order=2 \
                 cutoff=5.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=15 \
                 delta=1.0:\
     distance_Nb order=3 \
                 cutoff=4.0 \
                 covariance_type=ARD_SE \
                 theta_uniform=1.0 \
                 n_sparse=50 \
                 delta=0.004:\
     soap cutoff=4.0 \
          covariance_type=dot_product \
          zeta=2 \
          delta=0.016 \
          atom_sigma=0.7 \
          l_max=6 \
          n_max=6 \
          n_sparse=200 \
          sparse_method=cur_points} \
e0=-29.716948405885105 \
default_sigma={0.001 0.5 0.0 0.0} \
do_copy_at_file=F sparse_separate_file=F \
gp_file=/tmp/gap_2b3bsoap.xml 2>&1 | grep -v FoX

In [ ]:
gap_energies = []
for at in db:
    a = at.copy()
    a.set_calculator(gap_soap)
    gap_energies.append(a.get_potential_energy())

In [ ]:
np.sqrt(sum((np.array(qm_energies)/Natoms - np.array(gap_energies)/Natoms)**2)/len(gap_energies))

In [ ]:
from scipy import stats
sorted_energies = np.array(sorted(zip(qm_energies, gap_energies), key = lambda x: x[0]))
slope, intercept, r_value, p_value, std_err = stats.linregress(
    np.array(sorted_energies[:,0])/Natoms, np.array(sorted_energies[:,1])/Natoms
)

In [ ]:
plt.scatter(np.array(qm_energies)/Natoms, np.array(gap_energies)/Natoms)
plt.plot(np.array(sorted_energies[:,0])/Natoms, np.array(sorted_energies[:, 0])/Natoms * slope + intercept, 
         label = "slope = %.2f, fit = %.2f" % (round(slope, 5), round(r_value, 5)))
plt.xlabel("TB energies")
plt.ylabel("GAP energies")
# plt.xlim([-34.78, -34.775])
plt.legend()
plt.show()

In [ ]:
#attach gap calculator
atoms.set_calculator(gap_soap)
from ase.md.langevin import Langevin

timestep = 1.0 * units.fs
T = 1000

# Thermalize atoms
# MaxwellBoltzmannDistribution(atoms, 2.0* T * units.kB)

# dynamics = VelocityVerlet(atoms, timestep)
dynamics = Langevin(atoms, timestep, T * units.kB, 0.002)

dynamics.attach(print_status, interval=10)
dynamics.attach(print_energy, interval=10)
dynamics.run(steps=100)